# Superresolution

### This notebook introduces superresolution on the input images. The implementation applies diffusion by using Latent Diffusion model with a x4 upscaling factor. In this notebook, we will provide a step-by-step guide to apply superresolution to an image.

Select an input image

In [ ]:
from PIL import Image
import os, sys

input_path = '../inputs/superresolution/lenna.png'
input_image = Image.open(input_path)

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.path.dirname(current_dir), '..'))
sys.path.append(os.path.join(parent_dir, 'code'))

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(input_image)
plt.axis('off')
plt.show()

Define the number of inference steps

In [ ]:
steps = 50

Load and preapre the Latent Diffusion model

In [ ]:
from diffusers import LDMSuperResolutionPipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
pipeline = LDMSuperResolutionPipeline.from_pretrained("CompVis/ldm-super-resolution-4x-openimages")
pipeline = pipeline.to(device)

Since the model works only for the input shape of $128$ x $128$, we reisize the image to a width and length being a multiple of $128$

In [ ]:
low_res_img = input_image.convert("RGB")
original_width, original_height = low_res_img.size

if original_width > original_height:
    low_res_img = low_res_img.resize((768, 512))
    new_size = (768, 512)
elif original_width < original_height:
    low_res_img = low_res_img.resize((512, 768))
    new_size = (512, 768)
else:
    low_res_img = low_res_img.resize((512, 512))
    new_size = (512, 512)

As a next step the resized images is split into patches of shape $128$ x $128$

In [ ]:
patch_size = 128    

patches = []
img_width, img_height = low_res_img.size
for i in range(0, img_height, patch_size):
    for j in range(0, img_width, patch_size):
        box = (j, i, j + patch_size, i + patch_size)
        patch = low_res_img.crop(box)
        patches.append(patch)


Apply each of the patches to the model, and merge them back to obtain the full image

In [ ]:
from superres_ldm import reassemble_image

upscaled_patches = []
for patch in patches:
    upscaled_patch = pipeline(image=patch, num_inference_steps=steps, eta=1).images[0]
    upscaled_patches.append(upscaled_patch)

upscaled_image = reassemble_image(upscaled_patches, new_size[0], new_size[1], patch_size)
upscaled_image = upscaled_image.resize((original_width * 4, original_height * 4))

In [ ]:
plt.imshow(upscaled_image)
plt.axis('off')
plt.show()

Visalise the result

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(input_image)
axs[0].axis('off')
axs[0].set_title('Original image')

# Display the second image in the right subplot
axs[1].imshow(upscaled_image)
axs[1].axis('off')
axs[1].set_title('Final image')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()